In [167]:
import numpy as np
import pandas as pd

In [168]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [169]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [170]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [171]:
#Median 
train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].median())
test['LotFrontage'] = test['LotFrontage'].fillna(test['LotFrontage'].median())

In [172]:
#Mode
test.fillna({
    'MSZoning':test['MSZoning'].mode()[0],
    'Exterior1st': test['Exterior1st'].mode()[0],
    'Exterior2nd': test['Exterior2nd'].mode()[0],
    'KitchenQual': test['KitchenQual'].mode()[0],
    'Functional': test['Functional'].mode()[0],
    'SaleType': test['SaleType'].mode()[0]
}, inplace=True)
train.fillna({
    'Electrical':train['Electrical'].mode()[0]
}, inplace=True)


In [173]:
cols_with_none = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                   'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                   'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
                   'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
train[cols_with_none] = train[cols_with_none].fillna("None")
test[cols_with_none] = test[cols_with_none].fillna("None")

In [174]:
cols_with_0 = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
                'GarageCars', 'GarageArea']
train[cols_with_0] = train[cols_with_0].fillna(0)
test[cols_with_0] = test[cols_with_0].fillna(0)


In [175]:
cols_object = train.select_dtypes(include='object').columns
print(cols_object)
cols_objecttest = test.select_dtypes(include='object').columns
print(cols_objecttest)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 

In [176]:
train['SaleCondition'].unique()

array(['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family'],
      dtype=object)

In [177]:
#One Hot Encoding
train = pd.get_dummies(train, columns=['MSZoning', 'Street', 'Alley', 'LotShape',
                                       'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                                       'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                                       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                                       'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
                                       'Electrical','GarageType', 'Fence', 'MiscFeature',
                                       'SaleType', 'SaleCondition'],drop_first=True)
test = pd.get_dummies(test, columns=['MSZoning', 'Street', 'Alley', 'LotShape',
                                     'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                                     'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                                     'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
                                     'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
                                     'Electrical', 'GarageType', 'Fence', 'MiscFeature',
                                     'SaleType', 'SaleCondition'],drop_first=True)


In [178]:
# Ordinal mappings
mappings = {
    'ExterQual': {'Fa':0, 'TA':1, 'Gd':2, 'Ex':3},
    'ExterCond': {'Po':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
    'BsmtQual': {'None':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
    'BsmtCond': {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4},
    'BsmtExposure':{'No':0, 'None':0, 'Mn':1, 'Av':2, 'Gd':3},
    'BsmtFinType1':{'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
    'BsmtFinType2':{'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
    'HeatingQC':{'Po':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
    'CentralAir':{'N':0, 'Y':1},
    'KitchenQual': {'Fa':0, 'TA':1, 'Gd':2, 'Ex':3},
    'Functional':{'Typ':0, 'Min1':1, 'Min2':2, 'Mod':3, 'Maj1':4, 'Maj2':5, 'Sev':6},
    'FireplaceQu':{'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
    'GarageFinish':{'None':0, 'Unf':1, 'RFn':2, 'Fin':3},
    'GarageQual':{'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
    'GarageCond':{'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
    'PavedDrive':{'N':0, 'P':1, 'Y':2},
    'PoolQC':{'None':0, 'Fa':1, 'Gd':2, 'Ex':3}
}

for col, m in mappings.items():
    for df in [train, test]:
        df[col] = df[col].map(m)


In [179]:
test['KitchenQual']

0       1
1       2
2       1
3       2
4       2
       ..
1454    1
1455    1
1456    1
1457    1
1458    1
Name: KitchenQual, Length: 1459, dtype: int64

In [180]:
test['BsmtExposure'].unique()

array([0, 3, 1, 2])

In [181]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,2,...,False,False,False,False,True,False,False,False,True,False
1,2,20,80.0,9600,6,8,1976,1976,0.0,1,...,False,False,False,False,True,False,False,False,True,False
2,3,60,68.0,11250,7,5,2001,2002,162.0,2,...,False,False,False,False,True,False,False,False,True,False
3,4,70,60.0,9550,7,5,1915,1970,0.0,1,...,False,False,False,False,True,False,False,False,False,False
4,5,60,84.0,14260,8,5,2000,2000,350.0,2,...,False,False,False,False,True,False,False,False,True,False


In [182]:
test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,1,...,False,False,False,False,True,False,False,False,True,False
1,1462,20,81.0,14267,6,6,1958,1958,108.0,1,...,False,False,False,False,True,False,False,False,True,False
2,1463,60,74.0,13830,5,5,1997,1998,0.0,1,...,False,False,False,False,True,False,False,False,True,False
3,1464,60,78.0,9978,6,6,1998,1998,20.0,1,...,False,False,False,False,True,False,False,False,True,False
4,1465,120,43.0,5005,8,5,1992,1992,0.0,2,...,False,False,False,False,True,False,False,False,True,False
